In [1]:
import requests
import rasterio
import hvplot
import hvplot.xarray
import xarray as xr
import pandas as pd

import requests
from shapely.geometry import Point
import geopandas as gpd
from pyproj import CRS
import cartopy
import geoviews as gv

import warnings
warnings.filterwarnings('ignore')


### Pull in data from openaltimetry API

In [2]:
bounds = [-121.87893200997412, 48.7025807825158, -121.79541266948972, 48.7704252035505]

base_url = 'https://openaltimetry.org/data/api/icesat2/level3a'

payload =  {'product':'atl06',
            'startDate':'2018-10-14',
            'minx':str(bounds[0]),
            'miny':str(bounds[1]),
            'maxx':str(bounds[2]),
            'maxy':str(bounds[3]),
            'trackId':'326',
            'beamName':'gt2r',
            'outputFormat':'json'}

r = requests.get(base_url, params=payload)
elevation_data = r.json()

### Extract points and reproject to UTM zone 10 (EPSG:32610)

In [3]:
points = []            
for p in elevation_data['data'][0]['beams'][0]['lat_lon_elev']:
    points.append({
        'lat': p[0],
        'lon': p[1],
        'h': p[2]})
    
df = pd.DataFrame.from_dict(points)

geometry = [Point(xyz) for xyz in zip(df['lon'], df['lat'], df['h'])]        
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs=CRS('EPSG:4326'))
gdf = gdf.to_crs(32610)

x = []
y = []
for i in range(len(gdf)):
    x.append(gdf['geometry'].iloc[i].coords[:][0][0])
    y.append(gdf['geometry'].iloc[i].coords[:][0][1])
    


### Create geoviews point object from zipped list of x,y points in UTM zone 10 (EPSG:32610)

In [4]:
p = list(zip(x,y))

points = gv.Points(p, crs=cartopy.crs.UTM(10))

### Load DEM projected in UTM zone 10 (EPSG:32610)

In [5]:
dem_file_name = '/home/jovyan/shared/data-knuth/reference_dem_clip.tif'
src = rasterio.open(dem_file_name)
da = xr.open_rasterio(src)
img = da.sel(band=1).hvplot.image(rasterize=True,
                                  cmap='gray',
                                  geo=True)

### Plot points and raster together using holoviz syntax

In [6]:
img * points

:DynamicMap   []
   :Overlay
      .Image.I  :Image   [x,y]   (value)
      .Points.I :Points   [x,y]